In [1]:
import pandas as pd
import numpy as np
import random
import math
from itertools import permutations

In [2]:
# COUNT Completion Time of sequence of order in flowshop schedule
def count_completion_time(processing_times):
    process_accumlutaion = np.zeros(processing_times.shape)
    accumulation_time = 0
    for i in range(len(processing_times[0])):
        accumulation_time += processing_times[0][i]
        process_accumlutaion[0][i] = accumulation_time

    for i in range(1, len(processing_times)):
        accumulation_time = process_accumlutaion[i-1][0]
        for j in range(len(processing_times[i])):
            if(j == 0):
                accumulation_time += processing_times[i][j]
            else:
                if(process_accumlutaion[i-1][j] < process_accumlutaion[i][j-1]):
                    accumulation_time += processing_times[i][j]
                else:
                    accumulation_time = process_accumlutaion[i-1][j]
                    accumulation_time += processing_times[i][j]
            process_accumlutaion[i][j] = accumulation_time
    return accumulation_time

In [3]:
def generate_random_sequence(n):
  """Generates a random sequence of numbers from 1 to n."""
  sequence = random.sample(range(1, n+1), n)  # Sample 6 unique numbers from range 1 to 6
  return sequence # return list

In [4]:
def sequence_to_string(sequence):
  """Converts a sequence of numbers to a string.
  Args:
      sequence: A tuple of integers representing the sequence.

  Returns:
      A string representation of the sequence.
  """
  return '-'.join(map(str, sequence))

In [5]:

def generate_all_possibilites(data):

    numbers_of_job = len(data)
    numbers = range(1, numbers_of_job+1)  # Create a sequence from 1 to 6

    # Use permutations to generate all possible orderings
    seq = list(permutations(numbers))
    # Convert to List of String
    seq_str = [sequence_to_string(seq) for seq in seq]

    sol = dict()
    for x in range(len(seq)):
        seq_list = np.zeros(data.shape)
        for i in range(len(seq[x])):
            seq_list[i] = data[(seq[x][i])-1]
        sol.update({seq_str[x]: count_completion_time(seq_list)})
    return sol

In [6]:
def generate_random_solution(data, amount_of_solution):
    sol = dict()
    number_of_job = len(data)
    while len(sol) < amount_of_solution:  # Loop until desired number of solutions is reached
        seq = generate_random_sequence(number_of_job)
        seq_str = "-".join(map(str, seq))
        if seq_str not in sol:  # Check for uniqueness before adding to dictionary
            seq_list = np.zeros(data.shape)
            for i in range(len(seq)):
                seq_list[i] = data[(seq[i])-1]
            sol.update({seq_str: count_completion_time(seq_list)})
    return sol

In [7]:
# def generate_random_solution(data, amount_of_solution):
#     sol = dict()
#     number_of_job = len(data)-1
#     while len(sol) < amount_of_solution:  # Loop until desired number of solutions is reached
#         seq = generate_random_sequence(number_of_job)
#         print(seq)
#         seq_list = np.zeros(data.shape)
#         for i in range(len(seq)):
#             seq_list[i] = data[int(seq[i])-1]
#         sol.update({seq: count_completion_time(seq_list)})
#         # if seq not in sol:  # Check for uniqueness before adding to dictionary
#     return sol

In [8]:
def min_max_normalization(arr):
    min_val = np.min(arr) -1 # (np.min(arr)  * 1/100)
    max_val = np.max(arr)
    return (arr - min_val) / (max_val - min_val)

In [9]:
# Z-Score Normalization 
def zscore_normalization(arr):
    mean_val = np.mean(arr)
    std_dev = np.std(arr)
    return (arr - mean_val) / std_dev


In [10]:
def find_up_low_bound(arr):
    max_val = np.max(arr)
    min_val = np.min(arr)
    up_bound = math.ceil(max_val)
    low_bound = math.floor(min_val)
    return low_bound, up_bound

In [11]:
def sort_by_index(data):
  """Sorts a list and returns the corresponding sorted index sequence.

  Args:
      data: A list of sortable elements.

  Returns:
      A list of indices representing the sorted order of the original list.
  """
  return sorted(range(len(data)), key=lambda i: data[i])


### RUN

In [12]:
# Read The Data
# For data-0
# data = pd.read_csv("data/data0.csv")
# For data-1
data = pd.read_csv("data/data1.csv")
# For data-2
# data = pd.read_csv("data/data2.csv")
# For data-3
# data = pd.read_csv("data/data3.csv")

In [13]:
#LOAD DATA

data = data.values
data = data[:, 1:]
data

array([[135.62, 121.96, 3510.18, 2360.75, 213.38, 372.92],
       [90.45, 81.46, 2282.55, 1573.46, 142.3, 248.5],
       [197.82, 149.32, 430.24, 584.11, 3910.7, 237.15],
       [90.2, 81.43, 2282.08, 1573.88, 142.55, 248.02],
       [241.82, 215.66, 4335.05, 1839.83, 368.22, 378.59],
       [456.66, 2930.95, 464.22, 906.2, 119.37, 163.12],
       [135.17, 122.61, 3424.42, 2360.26, 213.85, 373.02],
       [240.74, 180.6, 4335.04, 1794.38, 1089.0, 387.12],
       [196.74, 148.52, 431.21, 561.37, 3809.12, 237.15],
       [194.14, 149.44, 427.29, 560.19, 3576.36, 237.15],
       [456.78, 2905.23, 463.52, 895.51, 134.61, 163.34],
       [195.39, 148.27, 448.22, 693.77, 3553.92, 237.15],
       [196.97, 149.88, 428.5, 574.35, 4005.38, 237.15],
       [240.94, 180.44, 4347.38, 1880.38, 364.75, 385.94],
       [196.8, 148.88, 432.35, 580.22, 3788.21, 237.15],
       [241.56, 183.71, 4351.61, 1811.75, 367.49, 362.58],
       [455.09, 2911.42, 462.47, 896.55, 215.36, 163.14]], dtype=object)

In [14]:
# INIT THE PARAMETERS
population_size = 10
iteration_ = 100
gamma = 0.01
alpha = 0.1
beta = 0.2
delta = 0.97

In [15]:
# Generate Random Solution based on population_size
solution_ = generate_random_solution(data=data, amount_of_solution=population_size)

In [16]:
# # RUN THIS FOR ALL POSSIBILITIES
# solution_ = generate_all_possibilites(data=data)

In [17]:
solution_

{'16-13-6-2-17-8-7-1-12-3-14-4-11-5-15-9-10': 45821.64000000001,
 '8-11-13-1-14-4-3-15-2-9-12-5-10-7-17-6-16': 38393.87,
 '6-7-13-15-16-14-5-2-4-11-9-3-12-10-8-17-1': 45628.80999999999,
 '11-10-8-16-6-2-1-12-3-14-7-15-5-9-13-17-4': 43478.090000000004,
 '5-3-13-16-12-15-6-9-8-14-2-1-17-10-4-7-11': 36404.01,
 '5-7-3-9-13-16-6-17-1-4-12-2-10-11-15-8-14': 37002.13000000001,
 '2-3-16-17-11-5-14-7-13-4-1-15-9-10-8-12-6': 45647.76000000001,
 '15-14-17-5-12-1-10-16-11-4-9-6-7-3-2-8-13': 39899.42,
 '12-5-17-1-6-10-9-8-2-4-15-11-16-13-7-14-3': 39382.09,
 '3-16-13-8-2-12-5-9-7-6-17-10-4-14-15-1-11': 36294.159999999996}

In [18]:
solution_keys = list(solution_.keys())
solution_list = list(solution_.values())
solution_list = np.array(solution_list)

In [19]:
solution_list

array([45821.64, 38393.87, 45628.81, 43478.09, 36404.01, 37002.13,
       45647.76, 39899.42, 39382.09, 36294.16])

In [20]:
normalize_solution = zscore_normalization(solution_list)

In [21]:
normalize_solution

array([ 1.33449444, -0.63754021,  1.28329907,  0.712294  , -1.16583778,
       -1.00704   ,  1.2883302 , -0.23782444, -0.37517289, -1.19500239])

In [22]:
low_bound, up_bound  = find_up_low_bound(normalize_solution)

In [23]:
# CREATE OBJECT AND RUN ALGORITHM
from firefly import firefly_algo

In [24]:
fa = firefly_algo(data=normalize_solution, population=population_size, max_iter=iteration_, low_bound=low_bound, up_bound=up_bound, gamma=gamma, alpha=alpha, beta=beta, delta=delta)
fa.run()

In [25]:
fa.print_result()


1.334
-45.999
1.258
0.712
-98.873
-82.511
1.257
-0.238
-25.547
-64.957


In [26]:
sorted_indices = sort_by_index(normalize_solution)

In [27]:
res = dict()
for i in sorted_indices:
    solution_key_list = f"{solution_keys[i]} - {solution_list[i]}";
    res.update({solution_key_list: fa.data[i]})

In [28]:
res

{'5-3-13-16-12-15-6-9-8-14-2-1-17-10-4-7-11 - 36404.01': -98.87294868050624,
 '5-7-3-9-13-16-6-17-1-4-12-2-10-11-15-8-14 - 37002.13000000001': -82.51100221473591,
 '3-16-13-8-2-12-5-9-7-6-17-10-4-14-15-1-11 - 36294.159999999996': -64.95680299182361,
 '8-11-13-1-14-4-3-15-2-9-12-5-10-7-17-6-16 - 38393.87': -45.999192672028386,
 '12-5-17-1-6-10-9-8-2-4-15-11-16-13-7-14-3 - 39382.09': -25.54673638958781,
 '15-14-17-5-12-1-10-16-11-4-9-6-7-3-2-8-13 - 39899.42': -0.23782444121716562,
 '11-10-8-16-6-2-1-12-3-14-7-15-5-9-13-17-4 - 43478.090000000004': 0.712293995550252,
 '2-3-16-17-11-5-14-7-13-4-1-15-9-10-8-12-6 - 45647.76000000001': 1.25735929071685,
 '6-7-13-15-16-14-5-2-4-11-9-3-12-10-8-17-1 - 45628.80999999999': 1.2575919297936344,
 '16-13-6-2-17-8-7-1-12-3-14-4-11-5-15-9-10 - 45821.64000000001': 1.3344944394264073}